# IMPORTING FILE

The data collection Android Application stores capacitive touch images as a flattend matrix in ".txt" files for each participant.

We will read all '.txt' file from individual participants. Then maps all field to pandas data frame and store them as pickle file. These pickle files will then be used for data pre-processing and augmentation. 

NOTE: This code is run on the local machine and the pkl files are uploaded because of memory constraints. 

In [ ]:
import sys
import os
import numpy as np
import pandas as pd

In [ ]:
def save_pickle (data,i):
    
    #For given participant convert data to Pandas data frame.
    df = pd.DataFrame(data, columns=['Participant', 'Handedness','Finger', 'Task', 'Timestamp', 'Matrix'])
    df.Timestamp = df.Timestamp.astype(int)
    save_path = 'D:/M.Sc. - INFOTECH/02. Summer Sem - 2018/Machine Learning - Lab/Dataset_Phalanx/P'+str(i)+'/data'+str(i)+'.pkl'
    df.to_pickle(save_path)

In [ ]:
data = list()
lines_skipped = 0

# Read data from all participant. In our case there are total 25 Participants.
for i in range(1,26):
    data = []
    DATA_PATH = 'D:/M.Sc. - INFOTECH/02. Summer Sem - 2018/Machine Learning - Lab/Dataset_Phalanx/P'+str(i)+'/'
    for fn in os.listdir(DATA_PATH):
        if os.path.isfile(DATA_PATH + fn):
            if (fn.startswith("CapMatrix") and fn.endswith(".txt")):
                print("Reading " + DATA_PATH + fn)
                lines = tuple(open(DATA_PATH + fn, 'r'))
                for l in lines:
                    l = l.split(";")
                    participant  = int(l[0])
                    hand = l[1]
                    finger = l[2]
                    task = l[3]
                
                    kernel_output = l[4] #Capacitive Matrix along with time stamp is kernel_output.
                    if (not ",," in kernel_output):
                        kernel_output = kernel_output.replace("\n", "")
                        kernel_output = kernel_output.split(",")
                        if (len(kernel_output) == 407):
                            kernel_output = kernel_output[:407]
                            time_ms = float(kernel_output[0])
                            time_ns = float(kernel_output[1])
                            timestamp = time_ms + time_ns//1000000000
                            m_arr = np.array(kernel_output[2:])
                            matrix = m_arr.astype(int)
                            data.append([participant, hand, finger, task, timestamp, matrix]) 
                        else:
                            lines_skipped = lines_skipped + 1
                    else:
                        lines_skipped = lines_skipped + 1   
        
        #Store current participant data to Pickle file in that folder.
        save_pickle(data,i)    
                                      
    
    